In [2]:
!pip install konlpy

In [3]:
import pandas as pd

In [4]:
df_review = pd.read_csv("../DataSet/ratings_train.txt", sep='\t')
df_review[:3]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [5]:
df_review['document'] = df_review['document'].str.replace('[0-9]','',regex=True)
df_review['document'] = df_review['document'].str.lower()
df_review['document'].str.findall('[A-Z]').value_counts()

document
[]    149995
Name: count, dtype: int64

In [6]:
df_review[:3]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [7]:
from konlpy.tag import Okt
okt = Okt()

In [8]:
def okt_clean(sentences):
  sentence_list = []
  for sentence in sentences :
    clean_text = []
    for word in okt.pos(sentence, stem=True):
        if word[1] not in ['Josa', 'Eomi', 'Punctuation','URL', 'Foreign']:
            clean_text.append(word[0])
            pass
    sentence_list.append(" ".join(clean_text))
  return sentence_list

In [9]:
df_review.dropna(subset=['document'], inplace=True)

In [10]:
Series_NLP = okt_clean(df_review['document'])
Series_NLP[:2]

['아 더빙 진짜 짜증나다 목소리', '흠 포스터 보고 초딩 영화 줄 오버 연기 가볍다 않다']

In [23]:
len(Series_NLP)

149995

In [24]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [22]:
df_review['document'][:2]

0                  아 더빙.. 진짜 짜증나네요 목소리
1    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
Name: document, dtype: object

In [21]:
# tokeninzing
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=None, ngram_range=(1,2))

In [23]:
tfidfVectorizer.fit(Series_NLP)

TfidfVectorizer(ngram_range=(1, 2))

### 모델 생성

In [13]:
train_target = df_review['label']
# train_feature = tfidfVectorizer.transform(Series_NLP)

In [29]:
from sklearn.ensemble import RandomForestClassifier
randomForestClassifier = RandomForestClassifier(n_jobs=-1)

In [30]:
model = randomForestClassifier.fit(X=train_feature, y=train_target)

In [11]:
import pickle

In [35]:
with open('movie_review.pkl','wb') as pickle_file :
  pickle.dump(obj=model, file=pickle_file)

In [12]:
file_path = 'movie_review.pkl'
with open(file_path, 'rb') as pickle_file:
  review_model = pickle.load(pickle_file)

### 모델 적용

In [14]:
df_test = df_review[:1500]
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, 0 to 1499
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1500 non-null   int64 
 1   document  1500 non-null   object
 2   label     1500 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 46.9+ KB


In [15]:
df_test[:3]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [16]:
df_test = df_test.copy()
df_test['document'] = df_test['document'].str.replace('[0-9]','',regex=True)
df_test['document'] = df_test['document'].str.lower()
df_test['document'].str.findall('[A-Z]').value_counts()

document
[]    1500
Name: count, dtype: int64

In [17]:
df_test.dropna(subset=['document'], inplace=True)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, 0 to 1499
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1500 non-null   int64 
 1   document  1500 non-null   object
 2   label     1500 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 46.9+ KB


In [18]:
Series_test = okt_clean(df_test['document'])
Series_test[:2]

['아 더빙 진짜 짜증나다 목소리', '흠 포스터 보고 초딩 영화 줄 오버 연기 가볍다 않다']

In [19]:
len(Series_test)

1500

In [28]:
test_target = df_test['label']

In [24]:
test_token = tfidfVectorizer.transform(Series_test)
test_token

<1500x569731 sparse matrix of type '<class 'numpy.float64'>'
	with 22675 stored elements in Compressed Sparse Row format>

In [25]:
test_model_array = review_model.predict(test_token.toarray())
list_token = test_model_array.tolist()
len(list_token)

1500

In [29]:
# score 방식 지정
from sklearn.metrics import f1_score, make_scorer,classification_report
scoring = make_scorer(f1_score)
print(classification_report(test_target,test_model_array))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       772
           1       0.99      0.98      0.99       728

    accuracy                           0.99      1500
   macro avg       0.99      0.99      0.99      1500
weighted avg       0.99      0.99      0.99      1500



### 표본 테스트

In [30]:
service_sentence = ["가능성만 있으면 어떤 기술로든 실현시키려는 게 인간의 본능인가봐."
                    ,"심해와도 같은 인간의 본질과 의식에 대해 잠겨본다."
                    ,"육체도 정신도 생명을 정의 할 수 없다 이 방대한 세상에서 나라는 존재를 어떻게 증명할것이며 나라는 의미는 어떻게 인식 해야하는가"
                    ,"전반적인 화려한 스케일과 액션, 캐릭터의 동작을 표현한 작화는 더할 나위 없이 좋았지만 그 속에 담긴 여러 철학을 온전히 이해하기까지는 지금의 제겐 아직 벅차 보입니다."
                    ,"내가 담기엔 영화가 너무 똑똑한가 보다."
                    ]

In [33]:
service_sentence = pd.Series(service_sentence)
service_sentence = service_sentence.str.replace('[0-9]','',regex=True)
service_sentence = service_sentence.str.lower()
service_sentence.str.findall('[A-Z]').value_counts()

[]    5
Name: count, dtype: int64

In [35]:
service_sentence.dropna()
service_sentence.info()

<class 'pandas.core.series.Series'>
RangeIndex: 5 entries, 0 to 4
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
5 non-null      object
dtypes: object(1)
memory usage: 172.0+ bytes


In [36]:
service_test = okt_clean(service_sentence)
service_test

['가능성 있다 어떻다 기술 로 실현 시키다 게 인간 본능 보다',
 '심해 같다 인간 본질 의식 대해 잠기다 보다',
 '육체 정신 생명 정의 하다 수 없다 이 방대하다 세상 나라 존재 어떻다 증명 하다 나라 의미 어떻다 인식 하다',
 '전반 적 화려하다 스케일 액션 캐릭터 동작 표현 작화 더하다 나위 없이 좋다 그 속 담기다 여러 철학 오다 이해 하다 지금 제겐 아직 벅차다 보이다',
 '내 담기다 영화 너무 똑똑하다 보다']

In [37]:
service_token = tfidfVectorizer.transform(service_test)
service_token

<5x569731 sparse matrix of type '<class 'numpy.float64'>'
	with 89 stored elements in Compressed Sparse Row format>

In [38]:
service_model_array = review_model.predict(service_token.toarray())
list_service_token = service_model_array.tolist()
list_service_token

[1, 1, 0, 1, 1]

In [39]:
# 비교 대상 : 리뷰 실제 별점 기준 2.5 이상 만족, 2.5 이하 불만족으로 반영
# 별점 4, 별점 3.5, 별점 4, 별점 3.5, 별점 1.0
service_target = [1, 1, 1, 1, 0]

In [40]:
print(classification_report(service_target,list_service_token))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.75      0.75      0.75         4

    accuracy                           0.60         5
   macro avg       0.38      0.38      0.38         5
weighted avg       0.60      0.60      0.60         5

